## Campos involucrados

- titulo
- descripcion

## Ideas

- wordcloud
- normalizacion
- stemming
- palabras positivas (respecto al precio)
- palabras negativas (respecto al precio)

## Hipótesis

- ciertas palabras indican mayor precio (luminoso, jardín, hermoso, vista...)
- a más palabras, mayor precio

## Resultados
- la correlacion entre longitud de descripcion y precio es bastante baja (0.1)
- la correlacion entre la cantidad de palabras positivas en la descripcion y el precio es bastante alta (0.3) [tener en cuenta que metrostotales tiene correlacion 0.5]

In [1]:
#importo las funciones para levantar los dataframes
%run "../../utils/dataset_parsing.ipynb"
#importo las funciones para graficar
%run "../../utils/graphs.ipynb"
df = levantar_datos("../../"+DATASET_RELATIVE_PATH)
df.columns
pd.set_option("display.max_colwidth", -1)

In [2]:
pd.set_option("display.max_columns", 50)

In [3]:
import nltk  
from nltk.corpus import stopwords  
from string import punctuation  

In [4]:
spanish_stopwords = set(stopwords.words('spanish'))
non_words = set(punctuation)
non_words.update({'¿', '¡'})
non_words.update(map(str,range(10)))

In [5]:
import re
from unidecode import unidecode

def is_meaningful(word: str) -> bool:
    """
        Recibe una palabra, remueve puntuaciones y verifica que lo que queda no esté en el set de stopwords
    """
    return len(word) > 2 and not word in spanish_stopwords

def remove_html(field: str) -> str:
    """
        Recibe un texto y devuelve una copia sin los tags html
    """
    return re.compile(r'<[^>]+>').sub('', field) if field else field

def normalize(field: str) -> str:
    """
        Recibe un texto y devuelve una copia sin acentos, ñ ni puntuaciones.
    """
    return ''.join([" " if c in non_words else unidecode(c) for c in field]).strip() if field else ""

def limpiar_campo(field: str) -> str:
    """
        Recibe un campo string que podría tener muchas palabras.
        Devuelve un string que contiene sólo las palabras significativas.
    """
    if not isinstance(field,str): return ""
    without_html = remove_html(field)
    normalized = normalize(without_html)
    meaningful = " ".join(set(filter(is_meaningful, normalized.split())))
    return meaningful

In [6]:
df["descripcion_limpia"] = df["descripcion"].map(limpiar_campo)
df["len_descripcion"] = df["descripcion_limpia"].map(lambda x: len(x.split()))

In [7]:
df["titulo_limpio"] = df["titulo"].map(limpiar_campo)
df["len_titulo"] = df["titulo_limpio"].map(lambda x: len(x.split()))

In [8]:
from collections import Counter

def get_word_counter(series):
    """
        Faltaría analizar stemming
    """
    counter = Counter()
    for title in series.values:
        counter.update(set(title.split()))
    return counter

In [9]:
titulo_palabras = get_word_counter(df["titulo_limpio"])
descripcion_palabras = get_word_counter(df["descripcion_limpia"])

In [10]:
print(len(titulo_palabras),len(descripcion_palabras))

18578 72121


In [11]:
# titulo_palabras.most_common(10)

In [12]:
# descripcion_palabras.most_common(10)

In [13]:
palabras_positivas = {"conservacion","tenis","balcon","panoramica","exclusivos","golf","canchas","remodelada","acondicionado","lujo","jacuzzi","diseno","exclusiva","magnifica","exclusivo","country","precioso","estilo","seguridad","verdes","juegos","servicio","excelente","terraza","jardin","hermosa","vista","bonita","renta", "granito"}
palabras_negativas = {"oportunidad","remato","oferta","remodelar"}

In [14]:
df["palabras_positivas_descripcion"] = df["descripcion_limpia"].map(lambda x: len([y for y in x.split() if y in palabras_positivas]))
df[["palabras_positivas_descripcion","precio"]].corr()

,palabras_positivas_descripcion,precio
palabras_positivas_descripcion,1.000000,0.338492
precio,0.338492,1.000000


In [15]:
df["palabras_negativas_descripcion"] = df["descripcion_limpia"].map(lambda x: len([y for y in x.split() if y in palabras_negativas]))
df[["palabras_negativas_descripcion","precio"]].corr()

,palabras_negativas_descripcion,precio
palabras_negativas_descripcion,1.000000,-0.026244
precio,-0.026244,1.000000


In [16]:
df.palabras_positivas_descripcion.value_counts()

1     64744
0     59049
2     46790
3     30454
4     18382
5     10290
6     5407 
7     2635 
8     1241 
9     571  
10    221  
11    98   
12    58   
14    31   
13    20   
15    8    
16    1    
Name: palabras_positivas_descripcion, dtype: int64

In [17]:
# df.loc[df.palabras_positivas_descripcion > 14]["descripcion"]

In [18]:
df.loc[df.palabras_negativas_descripcion > 2]["descripcion"]

6791      <p>residencia para remodelar. excelente ubicación frente a parque se incluyen planos de propuesta de remodelacion. oferta única en el mercado. casa habitación de tres recamaras con baño cada una. amplios espacios y terrazas. alberca de 6x3, cuarto de estudios. jardín, cochera para tres autos. oportunidad para inversionistas o desarrolladores.</p>                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
108031    <table class=adviewcontent cellpadding=0>\r\n<tbody>\r\n<tr>\r\n<td valign=top>remato casa en claveria, para remodelar, excelente oportunidad actualmente la construccion cuenta co

In [19]:
df_corr_positivas = df[["descripcion_limpia","precio"]]
for palabra in palabras_positivas:
    df_corr_positivas[palabra] = df_corr_positivas["descripcion_limpia"].map(lambda x: int(palabra in x))
df_corr_positivas.corr()["precio"].sort_values(ascending=False)

c:\users\dev\repos\fiuba\75-06\2019-2\tp\.venv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


precio           1.000000
terraza          0.223338
jardin           0.208592
vista            0.194534
jacuzzi          0.157660
lujo             0.155200
juegos           0.144366
servicio         0.132612
granito          0.102580
estilo           0.099340
hermosa          0.097818
exclusivo        0.091339
panoramica       0.087759
precioso         0.086908
balcon           0.081591
tenis            0.073820
excelente        0.065813
golf             0.064149
seguridad        0.060509
exclusiva        0.055698
diseno           0.052607
country          0.051681
remodelada       0.051611
exclusivos       0.040721
magnifica        0.039993
conservacion     0.028719
renta            0.026840
acondicionado    0.022942
canchas          0.021226
verdes          -0.022241
bonita          -0.040270
Name: precio, dtype: float64

In [20]:
df_corr_negativas = df[["descripcion_limpia","precio"]]
for palabra in palabras_negativas:
    df_corr_negativas[palabra] = df_corr_negativas["descripcion_limpia"].map(lambda x: int(palabra in x))
df_corr_negativas.corr()["precio"].sort_values(ascending=True)

c:\users\dev\repos\fiuba\75-06\2019-2\tp\.venv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


oportunidad   -0.042807
oferta        -0.031834
remato        -0.017778
remodelar      0.049695
precio         1.000000
Name: precio, dtype: float64

In [21]:
test = df[["descripcion_limpia","precio","metrostotales"]]
for palabra in palabras_positivas:
    test[palabra] = test["descripcion_limpia"].map(lambda x: int(palabra in x))

c:\users\dev\repos\fiuba\75-06\2019-2\tp\.venv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
top = list(set(test.corr()["metrostotales"].sort_values(ascending=False).head(8).index).union(set(test.corr()["precio"].sort_values(ascending=False).head(8).index)))

In [23]:
test_corr = test[top].corr()
test_corr["dif"] = test_corr["precio"] - test_corr["metrostotales"]
test_corr["dif"] = abs(test_corr["dif"])

In [24]:
test_corr["dif"].sort_values(ascending=False)
#estas se me ocurre que serian las palabras que mayor diferencia podrian hacer

metrostotales    0.485589
precio           0.485589
juegos           0.078725
lujo             0.077370
vista            0.070146
jacuzzi          0.052263
terraza          0.038639
jardin           0.025931
granito          0.006427
hermosa          0.004686
Name: dif, dtype: float64

In [25]:
test_corr

,juegos,granito,precio,vista,jacuzzi,lujo,jardin,terraza,metrostotales,hermosa,dif
juegos,1.000000,0.055081,0.144366,0.101708,0.109528,0.046619,0.146598,0.111850,0.065640,0.054155,0.078725
granito,0.055081,1.000000,0.102580,0.094475,0.040159,0.164585,0.148632,0.163145,0.109006,0.080282,0.006427
precio,0.144366,0.102580,1.000000,0.194534,0.157660,0.155200,0.208592,0.223338,0.514411,0.097818,0.485589
vista,0.101708,0.094475,0.194534,1.000000,0.094428,0.107811,0.124513,0.191207,0.124387,0.156207,0.070146
jacuzzi,0.109528,0.040159,0.157660,0.094428,1.000000,0.066057,0.102968,0.120713,0.105397,0.047128,0.052263
lujo,0.046619,0.164585,0.155200,0.107811,0.066057,1.000000,0.071130,0.113100,0.077831,0.067182,0.077370
jardin,0.146598,0.148632,0.208592,0.124513,0.102968,0.071130,1.000000,0.207999,0.234522,0.122408,0.025931
terraza,0.111850,0.163145,0.223338,0.191207,0.120713,0.113100,0.207999,1.000000,0.184700,0.096229,0.038639
metrostotales,0.065640,0.109006,0.514411,0.124387,0.105397,0.077831,0.234522,0.184700,1.000000,0.102505,0.485589
hermosa,0.054155,0.080282,0.097818,0.156207,0.047128,0.067182,0.122408,0.096229,0.102505,1.000000,0.004686


In [103]:
con_descripcion_y_titulo = df.loc[(df["len_descripcion"]>0) & (df["len_titulo"]>0)]

In [104]:
con_descripcion_y_titulo["titulo_descripcion"] = con_descripcion_y_titulo["titulo_limpio"] + "_" + con_descripcion_y_titulo["descripcion_limpia"]
con_descripcion_y_titulo["tiene_duplicado"] = con_descripcion_y_titulo["titulo_descripcion"].duplicated(keep=False)

c:\users\dev\repos\fiuba\75-06\2019-2\tp\.venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\dev\repos\fiuba\75-06\2019-2\tp\.venv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [87]:
duplicadas = con_descripcion_y_titulo.loc[con_descripcion_y_titulo["tiene_duplicado"]].sort_values(by="titulo_descripcion")

In [88]:
grouped = duplicadas.groupby(["titulo_descripcion"]).agg({"fecha":"nunique", "precio": ["nunique", "mean", "max", "min"]})

In [89]:
grouped.columns = [x+"_"+y for x,y in grouped.columns]

In [90]:
grouped["precio_dif"] = (grouped["precio_max"] - grouped["precio_min"]).astype(int)

In [91]:
grouped["precio_dif"].describe()

count    2.137000e+03
mean     3.185803e+05
std      7.299117e+05
min      0.000000e+00
25%      0.000000e+00
50%      7.000000e+04
75%      3.000000e+05
max      1.131212e+07
Name: precio_dif, dtype: float64

In [96]:
grouped.loc[grouped["precio_dif"]>grouped["precio_dif"].mean()].sort_values(by="precio_nunique", ascending=False)

,fecha_nunique,precio_nunique,precio_mean,precio_max,precio_min,precio_dif
titulo_descripcion,,,,,,
venta casa tijuana_lote manzana nocnok encuentra vivienda,11,38,5.366581e+05,820000.0,310808.0,509192
venta casa chihuahua_nocnok tipo credito contado venta posesion fisica ningun acepta exclusiva,6,16,4.929856e+05,895730.0,318696.0,577034
venta lote zapopan terreno_segundo canchas baja fut cine areas dos sauna semi residencial verdes terraza nivel multiusos olimpica lote voley basquet etapa club squash planta salon vestidores jogging alberca mas amplia vip sala fraccionamiento rapido tenis metros cancha juegos manzana cuenta vapor pista terreno arauca gimnasio casa,6,14,2.369864e+06,2913790.0,1823000.0,1090790
local placita sahuaros_tuboplus ventaneria concreto asfaltica cubiertas extractor excelente plaza tinaco norte enrollable medio instalacion adeblock ceramica guarniciones comercial agua ciudad aluminio amplia puerta carpeta bano aplanados marquesina metalica centimetros banquetas area aire cuenta nueva pisos local muros columnas estacionamiento fachada,4,13,7.737869e+05,1708350.0,479500.0,1228850
hipotecario chihuahua remate_diferencias explicamos litigiosos garantia inversion contado venta hipotecario realidad crediticios adjudicatarios superficies presentadas representar descripcion unicamente segura numeracion sabes media cesiones dedicada empresa pago avaluo derechos buenas imagenes sino condiciones registro aqui ubicacion debajo exactitud remate grandes tener publico notario pueden preocupes relacion oportunidades hipotecaria creditos inmobiliaria,2,13,8.024475e+05,1805692.0,334391.0,1471301
...,...,...,...,...,...,...
monte real_infonavitcasa buenas centro condiciones usar credito hospitales minutos ideal recamaras centros comerciales cerca casa,2,2,7.331000e+05,904000.0,562200.0,341800
monte depto huixquilucan remate col venta oferta jesus valor_contactenos apartado contado venta firma cita contrato cesion valor meses cel bancaria mayor departamento entrega pago inmobiliario derechos programar informacion recuperacion atte darle remate tels inversionistas publico real oferta notario consultor atencion creditos oficinas,2,2,1.075000e+06,1300000.0,850000.0,450000
modelo fincas privada_horno automoviles gas servicio canceles estacionario privada estufa techada equipada aspersion recamaras microondas planta integral lavavajillas bano cuarto jardin tanque riego campana calentador area banos cochera lavado casa cocina,2,2,3.720000e+06,4445000.0,2995000.0,1450000


In [100]:
con_descripcion_y_titulo.loc[con_descripcion_y_titulo["titulo_descripcion"]=="venta casa tijuana_lote manzana nocnok encuentra vivienda"].head(2)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,mes,ano,dia,descripcion_limpia,len_descripcion,titulo_limpio,len_titulo,palabras_positivas_descripcion,palabras_negativas_descripcion,tiene_descripcion,tiene_titulo,titulo_descripcion,tiene_duplicado
740,233323,casa en venta en tijuana,nocnok id: mx14-av4212. la vivienda se encuentra en el lote: 7; manzana: 404,Casa,NaN,Tijuana,Baja California Norte,NaN,2.0,NaN,2.0,69.0,123.0,NaN,32.549237,-116.947863,2015-04-16,False,False,False,False,False,706000.0,4,2015,16,lote manzana nocnok encuentra vivienda,5,venta casa tijuana,3,0,0,38,18,venta casa tijuana_lote manzana nocnok encuentra vivienda,True
12426,284591,casa en venta en tijuana,nocnok id: mx14-av3755. la vivienda se encuentra en el lote: 8; manzana: 816,Casa,NaN,Tijuana,Baja California Norte,NaN,2.0,NaN,2.0,68.0,127.0,2305.0,NaN,NaN,2015-05-15,False,False,False,False,False,601000.0,5,2015,15,lote manzana nocnok encuentra vivienda,5,venta casa tijuana,3,0,0,38,18,venta casa tijuana_lote manzana nocnok encuentra vivienda,True


In [101]:
con_descripcion_y_titulo.loc[con_descripcion_y_titulo["titulo_descripcion"]=="venta casa chihuahua_nocnok tipo credito contado venta posesion fisica ningun acepta exclusiva"].head(2)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,mes,ano,dia,descripcion_limpia,len_descripcion,titulo_limpio,len_titulo,palabras_positivas_descripcion,palabras_negativas_descripcion,tiene_descripcion,tiene_titulo,titulo_descripcion,tiene_duplicado
27050,189570,casa en venta en chihuahua,nocnok id: mx14-as2768. sin posesión física. no se acepta ningún tipo de crédito venta exclusiva al contado.,Casa,NaN,Chihuahua,Chihuahua,NaN,2.0,NaN,2.0,43.0,125.0,NaN,NaN,NaN,2016-03-11,False,False,False,False,False,339680.0,3,2016,11,nocnok tipo credito contado venta posesion fisica ningun acepta exclusiva,10,venta casa chihuahua,3,1,0,73,20,venta casa chihuahua_nocnok tipo credito contado venta posesion fisica ningun acepta exclusiva,True
52413,295344,casa en venta en chihuahua,nocnok id: mx14-as2769. sin posesión física. no se acepta ningún tipo de crédito venta exclusiva al contado.,Casa,NaN,Chihuahua,Chihuahua,NaN,2.0,NaN,2.0,69.0,120.0,15591.0,NaN,NaN,2016-02-09,False,False,False,False,False,895730.0,2,2016,9,nocnok tipo credito contado venta posesion fisica ningun acepta exclusiva,10,venta casa chihuahua,3,1,0,73,20,venta casa chihuahua_nocnok tipo credito contado venta posesion fisica ningun acepta exclusiva,True
